# Import #

In [61]:
from sklearn import svm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

# Lecture fichier #

In [2]:
test = pd.read_csv('test.csv',sep=',')
train = pd.read_csv('train.csv',sep=',')

# Scaling #

In [30]:
variables = train.columns.values
p = train.shape[1]
variables_cat = []
variables_bin = []
variables_other = []
for i in range(2,p):
    name = variables[i]
    if(name.find("cat")!=-1):
        variables_cat.append(name)
    elif(name.find("bin")!=-1):
        variables_bin.append(name)
    else:
        variables_other.append(name)

Mean = np.zeros(len(variables_other))
Std = np.zeros(len(variables_other))
for i in range(len(variables_other)):
    Mean[i] = np.mean(train[variables_other[i]])
    Std[i] = np.std(train[variables_other[i]])
    train[variables_other[i]] = (train[variables_other[i]]-Mean[i])/Std[i]
    test[variables_other[i]] = (test[variables_other[i]]-Mean[i])/Std[i]  #We scale the test_set with the same Mean and Std
    test[variables_other[i]] = (test[variables_other[i]]-Mean[i])/Std[i]

# Séparation pour Cross validation #

In [67]:
X = train.iloc[:,2:10].values
y = train.iloc[:,1].values
X_submit = test.iloc[:,1:9].values
X_submit_id = test.iloc[:,0].values

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.8, random_state = None)

# Classifier #

Rk: on demande une sortie en tant que probabilité donc on peut "abandonner" la target binaire. Ou répéter des classifications et faire de l'averaging
Rk: Le NN ne fait que prédire 0  ...
Donc accuracy a 0.9630 et les probabilités sont ridicukes à la fin...
RK: très faible proportion de mec ayant un 1: 3.6%
RK: Tester de sélectionner moins de paramètres
RK: Faire varier choix de loss 'mean_squared_error', 'binary_crossentropy'
RK: Faire varier le choix de la metrics 


In [78]:
rate = 0.001

classifier = Sequential()
classifier.add(Dense(units = 10, use_bias=True, kernel_initializer='random_uniform', bias_initializer='zeros', activation = 'relu', input_dim = 8))
classifier.add(Dropout(rate, noise_shape=None, seed=None))
classifier.add(Dense(units = 1, use_bias=True, kernel_initializer='random_uniform', bias_initializer='zeros', activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy']) 
classifier.fit(X_train, y_train, batch_size = 100, epochs = 5)

Epoch 1/5
119042/119042 [==============================] - 2s - loss: 0.0557 - acc: 0.9623     
Epoch 2/5
119042/119042 [==============================] - 2s - loss: 0.0360 - acc: 0.9631     
Epoch 3/5
119042/119042 [==============================] - 2s - loss: 0.0358 - acc: 0.9631     
Epoch 4/5
119042/119042 [==============================] - 2s - loss: 0.0357 - acc: 0.9631     
Epoch 5/5
119042/119042 [==============================] - 2s - loss: 0.0357 - acc: 0.9631     


# Normalized Gini coefficient #

Je peux essayer de transformer cette fonction pour qu'elle soit gérable par keras en tant que fonction à optimiser. Tensor has no arg len()

In [76]:
##TODO: Check the code
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    #assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
 
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
    
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

# Prediction #

In [63]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)
gini_value = gini_normalized(y_test,y_pred)
print("The Gini coefficient is: ", gini_value)

The Gini coefficient is:  -0.00140751453458


# Prediction finale #

In [64]:
y_pred_res = classifier.predict(X_submit)
#y_pred_submit = (y_pred_submit > 0.5)

#Create the Kaggle submission
res=[['id','target']]
for i in range(len(X_submit)):
    pair=[X_submit_id[i],y_pred_res[i][0]]
    res.append(pair)

# Ecriture du résultat #

In [65]:
submission = pd.DataFrame(res)
submission.to_csv('submission.csv', index=False, header=False) 


In [49]:
gini(y_pred,y_test)

/home/marc-jourdan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys


nan

0.036760135078375701

0.03636936388264695